# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The data represents 17 marketing campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
df.isnull().sum().sort_values().plot(kind = 'bar')
plt.savefig('images/missing_plot_.png')
plt.close();

y could be converted from object to bool.  Duration can be dropped.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Business Objective: With the knowledge of underlying demographic data, how can we target our marketing campaigns to achieve more success

In [6]:
bank_dict = ['age','job','marital','education','default','housing','loan','contact']
bank_to_encode = ['job','marital','education','default','housing','loan','contact']

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [7]:
bank_info = df[bank_dict]
bank_info = pd.get_dummies(bank_info[bank_to_encode])
X=bank_info
y=df['y']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [9]:
X1=X_train[:3000]
y1=y_train[:3000]
X2=X_test[:3000]
y2=y_test[:3000]

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

Using DummyClassifier from sklearn to set a baseline for comparison

In [69]:
%time
dummy_train = DummyClassifier().fit(X1, y1)
%time
dummy_test = DummyClassifier().fit(X2, y2)
print("train: ",dummy_train.score(X1,y1))
print("test: ", dummy_test.score(X2,y2))

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
CPU times: user 2 µs, sys: 14 µs, total: 16 µs
Wall time: 4.77 µs
train:  0.882
test:  0.8843333333333333


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [51]:
%time
lgr_train = LogisticRegression(multi_class='multinomial').fit(X1, y1)
%time
lgr_test = LogisticRegression(multi_class='multinomial').fit(X2, y2)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


### Problem 9: Score the Model

What is the accuracy of your model?

In [53]:
print("train: ",lgr_train.score(X1,y1))
print("test: ", lgr_test.score(X2,y2))

train:  0.8826666666666667
test:  0.8843333333333333


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [55]:
%time
knn_train = KNeighborsClassifier(n_neighbors=1).fit(X1, y1)
%time
knn_test = KNeighborsClassifier(n_neighbors=1).fit(X2, y2)
print("train: ",knn_train.score(X1,y1))
print("test: ", knn_test.score(X2,y2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs
train:  0.8546666666666667
test:  0.8603333333333333


In [57]:
%time
svc_train = SVC(kernel = 'linear').fit(X1, y1)
%time
svc_test = SVC(kernel = 'linear').fit(X2, y2)
print("train: ",svc_train.score(X1,y1))
print("test: ", svc_test.score(X2,y2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
train:  0.882
test:  0.8843333333333333


In [59]:
%time
dtc_train = DecisionTreeClassifier().fit(X1, y1)
%time
dtc_test = DecisionTreeClassifier().fit(X2, y2)
print("train: ",dtc_train.score(X1,y1))
print("test: ", dtc_test.score(X2,y2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs
train:  0.906
test:  0.906


In [61]:
res_dict = {'model': ['KNN', 'Logistic Regression', 'SVC','DecisionTree'],
           'train score': [.855, .883, .882,.906],
           'test score': [.86, .884, .884,.906],
           'average fit time': [6.08, 5.96, 4.17, 5.245]}
results_df = pd.DataFrame(res_dict).set_index('model')
results_df

,train score,test score,average fit time
model,,,
KNN,0.855,0.860,6.080
Logistic Regression,0.883,0.884,5.960
SVC,0.882,0.884,4.170
DecisionTree,0.906,0.906,5.245


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

Looking at various number of neighbors...comparing 1, 5, 9.  5 is a good balance of time and accuracy.

In [62]:
%time
knn_train = KNeighborsClassifier(n_neighbors=1).fit(X1, y1)
%time
knn_test = KNeighborsClassifier(n_neighbors=1).fit(X2, y2)
print("train: ",knn_train.score(X1,y1))
print("test: ", knn_test.score(X2,y2))

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 21 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
train:  0.8546666666666667
test:  0.8603333333333333


In [63]:
%time
knn_train = KNeighborsClassifier(n_neighbors=5).fit(X1, y1)
%time
knn_test = KNeighborsClassifier(n_neighbors=5).fit(X2, y2)
print("train: ",knn_train.score(X1,y1))
print("test: ", knn_test.score(X2,y2))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs
train:  0.8843333333333333
test:  0.884


In [64]:
%time
knn_train = KNeighborsClassifier(n_neighbors=9).fit(X1, y1)
%time
knn_test = KNeighborsClassifier(n_neighbors=9).fit(X2, y2)
print("train: ",knn_train.score(X1,y1))
print("test: ", knn_test.score(X2,y2))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs
train:  0.8873333333333333
test:  0.8846666666666667


Adjusting DecisionTree max_depth - assessing default,3, and 5.  3 is a good balance of time and accuracy

In [65]:
%time
dtc_train = DecisionTreeClassifier().fit(X1, y1)
%time
dtc_test = DecisionTreeClassifier().fit(X2, y2)
print("train: ",dtc_train.score(X1,y1))
print("test: ", dtc_test.score(X2,y2))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs
train:  0.906
test:  0.906


In [66]:
%time
dtc_train = DecisionTreeClassifier(max_depth=3).fit(X1, y1)
%time
dtc_test = DecisionTreeClassifier(max_depth=3).fit(X2, y2)
print("train: ",dtc_train.score(X1,y1))
print("test: ", dtc_test.score(X2,y2))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
train:  0.8826666666666667
test:  0.8853333333333333


In [67]:
%time
dtc_train = DecisionTreeClassifier(max_depth=5).fit(X1, y1)
%time
dtc_test = DecisionTreeClassifier(max_depth=5).fit(X2, y2)
print("train: ",dtc_train.score(X1,y1))
print("test: ", dtc_test.score(X2,y2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
train:  0.887
test:  0.887


Impacts of dropping gender feature...I couldn't find the GENDER feature so I decided to drop AGE instead and see what happened

In [71]:
bank_dict2 = ['job','marital','education','default','housing','loan','contact']
bank_to_encode = ['job','marital','education','default','housing','loan','contact']
bank_info2 = df[bank_dict2]
bank_info2 = pd.get_dummies(bank_info2[bank_to_encode])
XX=bank_info2
yy=df['y']
X_train2, X_test2, y_train2, y_test2 = train_test_split(XX, yy, random_state = 42)
X3=X_train[:3000]
y3=y_train[:3000]
X4=X_test[:3000]
y4=y_test[:3000]

In [72]:
%time
dummy_train2 = DummyClassifier().fit(X3, y3)
%time
dummy_test2 = DummyClassifier().fit(X4, y4)
print("train: ",dummy_train2.score(X3,y3))
print("test: ", dummy_test2.score(X4,y4))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
CPU times: user 2 µs, sys: 5 µs, total: 7 µs
Wall time: 7.15 µs
train:  0.882
test:  0.8843333333333333


In [73]:
%time
lgr_train2 = LogisticRegression(multi_class='multinomial').fit(X3, y3)
%time
lgr_test2 = LogisticRegression(multi_class='multinomial').fit(X4, y4)
print("train: ",lgr_train2.score(X3,y3))
print("test: ", lgr_test2.score(X4,y4))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
train:  0.8826666666666667
test:  0.8843333333333333


In [74]:
%time
knn_train2 = KNeighborsClassifier(n_neighbors=1).fit(X3, y3)
%time
knn_test2 = KNeighborsClassifier(n_neighbors=1).fit(X4, y4)
print("train: ",knn_train2.score(X3,y3))
print("test: ", knn_test2.score(X4,y4))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
train:  0.8546666666666667
test:  0.8603333333333333


In [75]:
%time
svc_train2 = SVC(kernel = 'linear').fit(X3, y3)
%time
svc_test2 = SVC(kernel = 'linear').fit(X4, y4)
print("train: ",svc_train2.score(X3,y3))
print("test: ", svc_test2.score(X4,y4))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
train:  0.882
test:  0.8843333333333333


In [76]:
%time
dtc_train2 = DecisionTreeClassifier().fit(X3, y3)
%time
dtc_test2 = DecisionTreeClassifier().fit(X4, y4)
print("train: ",dtc_train2.score(X3,y3))
print("test: ", dtc_test2.score(X4,y4))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs
train:  0.906
test:  0.906


In [77]:
res_dict2 = {'model': ['KNN', 'Logistic Regression', 'SVC','DecisionTree'],
           'train score': [.855, .883, .882,.906],
           'test score': [.86, .884, .884,.906],
           'average fit time': [5.49, 5.61, 5, 6.2]}
results_df2 = pd.DataFrame(res_dict2).set_index('model')
results_df2

,train score,test score,average fit time
model,,,
KNN,0.855,0.860,5.49
Logistic Regression,0.883,0.884,5.61
SVC,0.882,0.884,5.00
DecisionTree,0.906,0.906,6.20


In [78]:
results_df

,train score,test score,average fit time
model,,,
KNN,0.855,0.860,6.080
Logistic Regression,0.883,0.884,5.960
SVC,0.882,0.884,4.170
DecisionTree,0.906,0.906,5.245


A quick look shows no change in test/train scores (keeping the default arguments instead of the optimized ones) with a diverging change in fit time.  KNN & LGR both took longer on average while SVC and DecisionTree sped up when the AGE feature was removed.